In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import torchvision.io as io
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
model = models.resnet50(pretrained=True, progress=True)

In [3]:
# fine tune the last layer
model.fc = nn.Linear(in_features=2048, out_features=8, bias=True)

In [4]:
augment = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.405], std=[0.229, 0.224, 0.225])
])

In [5]:
class Data(torch.utils.data.Dataset):
    def __init__(self, augment):
        self.data = []
        self.labels = []
        self.dir = 'D:/Data/hd/train'
        self.augment = augment
        
        i = 0
        for folder in os.listdir(self.dir):
            label = i
            for file in tqdm(os.listdir(os.path.join(self.dir, folder))):
                img_path = os.path.join(os.path.join(self.dir, folder), file)
                
                # Open and apply augmentation
                img = io.read_image(img_path)
                img = self.augment(img)
                 
                # Append the image and its corresponding label to the output
                self.data.append(img)
                self.labels.append(label)
            i += 1
            
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [6]:
data = Data(augment)

100%|███████████████████████████████████████████████| 3210/3210 [00:49<00:00, 65.20it/s]


In [7]:
device = torch.device('cuda')

In [8]:
model = model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
loss = nn.CrossEntropyLoss()

In [9]:
train, val = torch.utils.data.random_split(data, [22972, 2552])
trainloader = torch.utils.data.DataLoader(train, shuffle=True, batch_size=32)
valloader = torch.utils.data.DataLoader(val, shuffle=True, batch_size=32)

In [10]:
training_losses, val_losses = [], []
avg_training_losses, avg_val_losses = [], []
accuracies = []

In [ ]:
for e in tqdm(range(7)):
    for x, y in trainloader:
        model.train()
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        y_pred = model(x.float())
        l = loss(y_pred, y)
        l.backward()
        optimizer.step()
        npl = l.detach().cpu().numpy()
        training_losses.append(npl)

    acc = 0
    acc_t, acc_p = [], []
    with torch.no_grad():
        for x, y in valloader:
            model.eval()
            x, y = x.to(device), y.to(device)
            y_pred = model(x.float())
            l = loss(y_pred, y)
            npl = l.detach().cpu().numpy()
            val_losses.append(npl)
            
            # accuracy metric as well
            p = np.argmax(y_pred.detach().cpu().numpy(), axis=1)
            tp = y.detach().cpu().numpy()
            acc_p.append(p)
            acc_t.append(tp)
    
    for x, y in zip(acc_p, acc_t):
        for i, j in zip(x, y):
            if i == j:
                acc += 1
                
    val_acc = acc / 2552
    avg_training_losses.append(np.mean(training_losses))
    avg_val_losses.append(np.mean(val_losses))
    accuracies.append(val_acc)

    print(f'Epoch: {e}, Training Loss: {np.mean(training_losses)}, Validation Loss: {np.mean(val_losses)}, Accuracy: {val_acc}')

  4%|█▉                                               | 1/25 [03:04<1:13:38, 184.10s/it]

Epoch: 0, Training Loss: 1.3199018239974976, Validation Loss: 1.0639394521713257, Accuracy: 0.6344043887147336


  8%|███▉                                             | 2/25 [06:06<1:10:15, 183.30s/it]

Epoch: 1, Training Loss: 1.1456713676452637, Validation Loss: 1.0228689908981323, Accuracy: 0.670846394984326


 12%|█████▉                                           | 3/25 [09:11<1:07:21, 183.70s/it]

...

Epoch: 14, Training Loss: 0.5347082018852234, Validation Loss: 0.8402426242828369, Accuracy: 0.7480407523510971


 64%|████████████████████████████████                  | 16/25 [49:05<27:39, 184.41s/it]

Epoch: 15, Training Loss: 0.5107452869415283, Validation Loss: 0.8515939712524414, Accuracy: 0.7503918495297806


 68%|██████████████████████████████████                | 17/25 [52:10<24:35, 184.40s/it]

Epoch: 16, Training Loss: 0.48894014954566956, Validation Loss: 0.858160138130188, Accuracy: 0.7621473354231975


In [ ]:
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'loss': loss
}, 'resnet50baseline.pth')